In [1]:
# -*- coding: utf-8 -*-
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import tensorflow as tf
import struct 
import numpy as np

from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
import argparse

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### 这是客户端本地项目路径  映射到   /code

In [3]:
!ls

LICENSE
README.md
Untitled-1
Untitled-1p
Untitled.ipynb
bug-fix.ipynb
ci
download_data.py
latest_helper.yml
linear_regression.1.ipynb
linear_regression.house.price.ipynb
lnv_build.yml
log
lr-13.png
mnist1
model.ops
model.py
model_tf.py
modelp
polyaxonfile.yml
polyaxonfile_declarations.yml
polyaxonfile_hyperparams.yml
polyaxonfile_hyperparams_bo.yml
polyaxonfile_hyperparams_early_stopping.yml
polyaxonfile_hyperparams_grid.yml
polyaxonfile_hyperparams_hyperband.yml
polyaxonfile_job.yml
polyaxonfile_notebook.yml
polyaxonfile_tensorboard.yml
predict.ckpt.py
predict.es.ipynb
predict.es.py
sgd_classifier.ipynb
t.ipynb
test.py
testcomment.py
train.es.ipynb


#### 先下载数据库
     客户端执行  polyaxon run -f job.yml  然后会在  虚拟主机/data/mnist在有数据文件

####  数据路径存在虚拟主机路径下： /data/mnist 注意，客户端需要执行 polyaxon run -f job.yml

In [4]:
!ls /data/mnist
data_path = '/data/mnist/'

t10k-images-idx1-ubyte	   t10k-labels-idx1-ubyte.gz
t10k-images-idx3-ubyte	   train-images-idx3-ubyte.gz
t10k-images-idx3-ubyte.gz  train-labels-idx1-ubyte.gz
t10k-labels-idx1-ubyte


In [5]:
def set_logging(log_level=None):
    if log_level == 'INFO':
        log_level = tf.logging.INFO
    elif log_level == 'DEBUG':
        log_level = tf.logging.DEBUG
    elif log_level == 'WARN':
        log_level = tf.logging.WARN
    else:
        log_level = 'INFO'

    tf.logging.set_verbosity(log_level)

In [6]:
set_logging('DEBUG')


In [7]:
#加载数据
mnist = input_data.read_data_sets(data_path, one_hot=False)

Extracting /data/mnist/train-images-idx3-ubyte.gz
Extracting /data/mnist/train-labels-idx1-ubyte.gz
Extracting /data/mnist/t10k-images-idx3-ubyte.gz
Extracting /data/mnist/t10k-labels-idx1-ubyte.gz


In [8]:
#定义模型网络  tf.esstimator api
def get_model_fn(learning_rate, dropout, activation):
    """Create a `model_fn` compatible with tensorflow estimator based on hyperparams."""

    def get_network(x_dict, is_training):
        with tf.variable_scope('network'):
            x = x_dict['images']
            x = tf.reshape(x, shape=[-1, 28, 28, 1])
            conv1 = tf.layers.conv2d(x, 32, 5, activation=activation)
            conv1 = tf.layers.max_pooling2d(conv1, 2, 2)
            conv2 = tf.layers.conv2d(conv1, 64, 3, activation=activation)
            conv2 = tf.layers.max_pooling2d(conv2, 2, 2)
            fc1 = tf.contrib.layers.flatten(conv2)
            fc1 = tf.layers.dense(fc1, 1024)
            fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)
            out = tf.layers.dense(fc1, 10)
        return out

    def model_fn(features, labels, mode):
        is_training = mode == tf.estimator.ModeKeys.TRAIN

        results = get_network(features, is_training=is_training)

        predictions = tf.argmax(results, axis=1)

        # Return prediction
        if mode == tf.estimator.ModeKeys.PREDICT:
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)

        # Define loss
        loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=results, labels=tf.cast(labels, dtype=tf.int32)))
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())

        # Evaluation metrics
        accuracy = tf.metrics.accuracy(labels=labels, predictions=predictions)
        precision = tf.metrics.precision(labels=labels, predictions=predictions)

        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions=predictions,
            loss=loss_op,
            train_op=train_op,
            eval_metric_ops={'accuracy': accuracy, 'precision': precision})

    return model_fn




In [9]:
#模型参数
batch_size =128
num_steps =800
num_iterations = 1
num_epochs = 1

learning_rate = 0.001
dropout = 0.25
activation = 'relu'
distributed = False



In [10]:
#模型保存路径，没有新建，，这个是在pod内的，没有持久化
outputs_path = './log'

In [19]:
!ls /outputs/root/quick-start1/

experiments  groups  jobs


In [21]:
!ls /outputs/root/quick-start1/experiments/5

checkpoint
eval
events.out.tfevents.1553656058.plx-d9f55b96b11b470083a5362839007717-master-0
graph.pbtxt
model.ckpt-1.data-00000-of-00001
model.ckpt-1.index
model.ckpt-1.meta
model.ckpt-430.data-00000-of-00001
model.ckpt-430.index
model.ckpt-430.meta


In [25]:
#虽然看不到虚拟主机的目录，但是pod根目录下的outputs 是挂在虚拟主机的 /mnt/polyaxon/outputs这个目录的，若新建，会同步过去的
#同步到虚拟主机后，这样pod失效，也不会影响虚拟主机的中保存的模型，并且可一 scp 拷贝到本地
!ls /mnt/polyaxon/outputs

ls: cannot access '/mnt/polyaxon/outputs': No such file or directory


In [27]:
!mkdir /outputs/root/quick-start1/experiments/ipython
!ls /outputs/root/quick-start1/experiments/

mkdir: cannot create directory '/outputs/root/quick-start1/experiments/ipython': File exists
3  4  5  ipython


In [28]:


#模型
outputs_path_vm = '/outputs/root/quick-start1/experiments/ipython'

In [29]:
if activation == 'relu':
    activation = tf.nn.relu
elif activation == 'sigmoid':
    activation = tf.nn.sigmoid
elif activation == 'linear':
    activation = None

In [31]:
#模型实例
estimator = tf.estimator.Estimator(
    get_model_fn(learning_rate=learning_rate, dropout=dropout, activation=activation),
    model_dir=outputs_path_vm)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': 'worker', '_tf_random_seed': None, '_master': '', '_num_ps_replicas': 0, '_service': None, '_session_config': None, '_model_dir': '/outputs/root/quick-start1/experiments/ipython', '_is_chief': True, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f44b8b1dd30>, '_save_summary_steps': 100, '_task_id': 0, '_num_worker_replicas': 1, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000}


In [32]:
#开始训练,在虚拟机里的环境训练

input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.train.images},
    y=mnist.train.labels,
    batch_size=batch_size,
    num_epochs=num_epochs,
    shuffle=True)

In [33]:
for i in range(num_iterations):
    estimator.train(input_fn, steps=num_steps)

    # Evaluate the Model
    input_fn = tf.estimator.inputs.numpy_input_fn(
        x={'images': mnist.test.images},
        y=mnist.test.labels,
        batch_size=batch_size,
        shuffle=False)

    metrics = estimator.evaluate(input_fn)

    print("Testing metrics: {}", metrics)
    print('loss=:{},accuracy={},precision={}'.format( 
        metrics['loss'],
                           metrics['accuracy'],
                           metrics['precision']))

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /outputs/root/quick-start1/experiments/ipython/model.ckpt.
INFO:tensorflow:step = 1, loss = 2.3205762
INFO:tensorflow:global_step/sec: 11.2814
INFO:tensorflow:step = 101, loss = 0.10559823 (8.865 sec)
INFO:tensorflow:global_step/sec: 12.9955
INFO:tensorflow:step = 201, loss = 0.07940368 (7.695 sec)
INFO:tensorflow:global_step/sec: 11.091
INFO:tensorflow:step = 301, loss = 0.031227576 (9.016 sec)
INFO:tensorflow:global_step/sec: 12.1377
INFO:tensorflow:step = 401, loss = 0.03772916 (8.239 sec)
INFO:tensorflow:Saving checkpoints for 430 into /outputs/root/quick-start1/experiments/ipython/model.ckpt.
INFO:tensorflow:Loss for final step: 0.06753243.
INFO:tensorflow:Starting evaluation at 2019-03-30-03:21:52
INFO:tensorflow:Restoring parameters from /outputs/root/quick-start1/experiments/ipython/model.ckpt-430
INFO:tensorflow:Finished evaluation at 2019-03-30-03:21:54
INFO:tensorflow:Saving dict for gl

In [32]:
#这个
!ls ./log/

checkpoint
eval
events.out.tfevents.1553700970.zkl-MACH-WX9
events.out.tfevents.1553752727.plx-notebook-d0507202fec244d2a7dd8ece3c8d078c-fbd79c85f-jtn4j
graph.pbtxt
model.ckpt-0.data-00000-of-00001
model.ckpt-0.index
model.ckpt-0.meta
model.ckpt-430.data-00000-of-00001
model.ckpt-430.index
model.ckpt-430.meta
model.ckpt-431.data-00000-of-00001
model.ckpt-431.index
model.ckpt-431.meta
model.ckpt-860.data-00000-of-00001
model.ckpt-860.index
model.ckpt-860.meta


In [34]:
!ls {outputs_path_vm}

checkpoint
eval
events.out.tfevents.1553916075.plx-notebook-58e143aa2f7f4bb384b4cc23f075fc5a-665f9dcf9f-kjzqm
graph.pbtxt
model.ckpt-1.data-00000-of-00001
model.ckpt-1.index
model.ckpt-1.meta
model.ckpt-430.data-00000-of-00001
model.ckpt-430.index
model.ckpt-430.meta
